In [50]:
# import sys
# sys.path.append('..')

# Data Extraction/Scrapping

Note: This script will not give the all 10-K FORM of all 2568 URLs at a single shot. Give only 1000 URLs at time.

In [51]:
import re
import time
import requests
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import urllib.request as url
from bs4 import BeautifulSoup as bs
import pandas as pd

In [52]:
%%time
#from google.colab import drive
#drive.mount('/content/drive')

CPU times: total: 0 ns
Wall time: 0 ns


In [53]:
phd_data = pd.read_csv("../csv/10k_filing_info_train_links-1570100710859.csv")

In [54]:
phd_data.head()

,ticker,cik,filing_date,10k_link,long_term_outlook
0,FE,1031296,2012-02-28,https://www.sec.gov/Archives/edgar/data/103129...,0
1,CL,21665,2015-02-19,https://www.sec.gov/Archives/edgar/data/21665/...,0
2,PRU,1137774,2016-02-19,https://www.sec.gov/Archives/edgar/data/113777...,1
3,EBAY,1065088,2001-03-28,https://www.sec.gov/Archives/edgar/data/106508...,1
4,CAM,941548,2005-02-25,https://www.sec.gov/Archives/edgar/data/941548...,1


In [55]:
ticker_counts = phd_data['ticker'].value_counts()
ticker_counts_filtered = ticker_counts[:1]
# ticker_counts_filtered = ticker_counts[ticker_counts > 20]
ticker_counts_filtered

TMK    23
Name: ticker, dtype: int64

In [56]:
# tmk_data = phd_data[phd_data['ticker'] == 'TMK']
# tmk_data.sort_values(by='filing_date')

In [57]:
all_link = phd_data['10k_link']
all_cik = phd_data['cik']
all_ticker = phd_data['ticker']
all_filing_date = phd_data["filing_date"]
all_long_term_outlook = phd_data["long_term_outlook"]

all_data  = list(zip(all_ticker,all_cik,all_filing_date,all_long_term_outlook,all_link))

all_data = [
    (ticker, cik, filing_date, long_term_outlook, link)
    for ticker, cik, filing_date, long_term_outlook, link in all_data
    if ticker in ticker_counts_filtered.index
]

In [58]:
len(all_data)

23

In [59]:
all_data[0]

('TMK',
 320335,
 '2000-03-23',
 1,
 'https://www.sec.gov/Archives/edgar/data/320335/0000931763-00-000595.txt')

In [60]:
#type(all_link)

In [61]:
URL = "https://www.sec.gov/Archives/edgar/data/1031296/000103129612000012/0001031296-12-000012.txt"
#url = "https://www.sec.gov/Archives/edgar/data/1031296/000103129612000012/0001031296-12-000012.txt"

In [62]:
URL = ["https://www.sec.gov/Archives/edgar/data/1031296/000103129612000012/0001031296-12-000012.txt","https://www.sec.gov/Archives/edgar/data/21665/000162828015000846/0001628280-15-000846.txt"]

In [63]:
data_10k = {"ticker":None,"cik":None,"filing_date":None,"sentence":None,"long_term_outlook":None}

In [64]:
data_10k

{'ticker': None,
 'cik': None,
 'filing_date': None,
 'sentence': None,
 'long_term_outlook': None}

In [65]:
def get_url_data(URL):
  try:
    print ("executing URL",URL[4])
    time.sleep(5)
    data = url.urlopen(URL[4]).read()
    print("data: ", data)
    soup = bs(data,'html.parser')
    print("soup: ", soup)
    doc = soup.find_all("type")
    doc = soup.find(name='body')
    print("found doc: ", doc) 
    return (doc,URL[0:4])
  except:
    print ("crashed for the URL",URL)
    #doc = soup.find(name='body')
    return (None,URL[0:4])

In [66]:
def scrap_data(doc,xdata):
  main_data = []
  for i in doc:
    cont = i.contents
    if cont[0].strip() =="10-K":
      print ("Found")
      main_data.append(cont[1].text)
  return (main_data,xdata)


In [67]:
#doc = get_url_data(URL)

In [68]:
#d = scrap_data(doc)
#{'ticker': None, 'cik': None, 'filing_date': None, 'sentence': None}

In [69]:
def write_data(d,x):
    if data_10k.get('sentence'):
        ts = data_10k.get('sentence')
        tk = data_10k.get('ticker')
        tcik = data_10k.get('cik')
        tdate = data_10k.get('filing_date')
        tt = data_10k.get('long_term_outlook')
    else:
        ts,tk,tcik,tdate,tt = [],[],[],[],[]
    ts.append(d)
    tk.append(x[0])
    tcik.append(x[1])
    tdate.append(x[2])
    tt.append(x[3])
    data_10k['sentence'] = ts.copy()
    data_10k['ticker'] = tk.copy()
    data_10k['cik'] = tcik.copy()
    data_10k['filing_date'] = tdate.copy()
    data_10k['long_term_outlook'] = tt.copy()
    print("Sucessfully Write!!..")

In [70]:
# write_data(d)all_data

In [71]:
all_data

[('TMK',
  320335,
  '2000-03-23',
  1,
  'https://www.sec.gov/Archives/edgar/data/320335/0000931763-00-000595.txt'),
 ('TMK',
  320335,
  '1999-03-12',
  0,
  'https://www.sec.gov/Archives/edgar/data/320335/0000931763-99-000710.txt'),
 ('TMK',
  320335,
  '2006-03-06',
  1,
  'https://www.sec.gov/Archives/edgar/data/320335/000119312506046266/0001193125-06-046266.txt'),
 ('TMK',
  320335,
  '2012-02-28',
  1,
  'https://www.sec.gov/Archives/edgar/data/320335/000119312512085031/0001193125-12-085031.txt'),
 ('TMK',
  320335,
  '1997-03-25',
  1,
  'https://www.sec.gov/Archives/edgar/data/320335/0000950109-97-002410.txt'),
 ('TMK',
  320335,
  '1994-03-25',
  0,
  'https://www.sec.gov/Archives/edgar/data/320335/0000950109-94-000538.txt'),
 ('TMK',
  320335,
  '2011-02-28',
  1,
  'https://www.sec.gov/Archives/edgar/data/320335/000119312511049737/0001193125-11-049737.txt'),
 ('TMK',
  320335,
  '2009-02-27',
  0,
  'https://www.sec.gov/Archives/edgar/data/320335/000119312509040782/00011931

In [72]:
%%time

c=0
# for u in all_data[0:1000]:#instead of passing single URL passed 8
for u in all_data[:2]:
  print("Count:{0}".format(c))  
  doc, xdata = get_url_data(u)
  print("doc",doc)
  if doc:
    d,x = scrap_data(doc,xdata)
    #print(d)
    write_data(d,x)
  else:
    print("data crashed")
    #write_data("data crashed")
  c += 1

Count:0
executing URL https://www.sec.gov/Archives/edgar/data/320335/0000931763-00-000595.txt


crashed for the URL ('TMK', 320335, '2000-03-23', 1, 'https://www.sec.gov/Archives/edgar/data/320335/0000931763-00-000595.txt')
doc None
data crashed
Count:1
executing URL https://www.sec.gov/Archives/edgar/data/320335/0000931763-99-000710.txt
crashed for the URL ('TMK', 320335, '1999-03-12', 0, 'https://www.sec.gov/Archives/edgar/data/320335/0000931763-99-000710.txt')
doc None
data crashed
CPU times: total: 15.6 ms
Wall time: 10.2 s


In [73]:
#%%time
#for u in all_link[1:11]:#instead of passing single URL passed 8
#  doc = get_url_data(u)
#  if doc:
#    d = scrap_data(doc)
#    #print(d)
##    write_data(d)
#  else:
#    print("data crashed")
#    #write_data("data crashed")

In [74]:
ar = pd.DataFrame(data_10k)


KeyboardInterrupt



In [ ]:
ar.head(10)

NameError: name 'ar' is not defined

In [ ]:
ar.to_csv("final_updated_1000.csv",index = False)